In [2]:
! pip install -qU gigachain==0.2.6 gigachain_community==0.2.6 gigachain-cli==0.0.25 langgraph==0.2.19 langchain-community==0.2.16 langchain-openai==0.1.23  python-pptx==0.6.23 python-pptx-interface==0.0.12 python-dotenv==1.0.1 duckduckgo-search==6.2.4 faiss-cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.48.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.48.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.48.0 requires pyarrow<12.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
langchain-mistralai 0.2.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.2.41 which is incompatible.
patent-client 4.1.10 requires httpx[http2]<0.26.0,>=0.25.2, but you have httpx 0.27.0 which is incompatible.
python-telegram-bot 20.3 requires httpx~=0.24.0, but you have httpx 0.27.0 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.
types-requests 2.32.0.20241016 requires urllib3>=2, but you have urllib3 1.26.20 which is inco

In [1]:
import getpass
import os
import requests
import json
import time

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings

from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool

# import faiss
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from pydantic import BaseModel

In [2]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('.env')

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [4]:
class SimpleChatBot:
    def __init__(self, llm, memory_type, **memory_kwargs):
        #TODO
        if memory_type == 'summary':
            self.memory = ConversationSummaryMemory(llm=llm, **memory_kwargs)
        elif memory_type == 'simple':
            self.memory = ConversationBufferMemory(**memory_kwargs)
        elif memory_type == 'window':
            self.memory = ConversationBufferWindowMemory(**memory_kwargs)
            
        self.conversation = ConversationChain(
            llm=llm,
            verbose=True,
            memory=self.memory
        )

    def _respond(self, user_input):
        response = self.conversation.predict(input=user_input)
        return response
    
    def print_memory(self):
        print(self.memory.load_memory_variables({}))

    def run(self):
        print("This is a simple chat bot:")
        while True:
            user_input = input("You: ")
            if user_input.lower() == "bye":
                break
            response = self._respond(user_input)
            print(f"Bot: {response}")
        print("Chat ended.")

Now let's check how it works with each type of memory

In [5]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary', memory_window_size=2)
chat.run()
chat.print_memory()

C:\Users\Tseh\AppData\Local\Temp\ipykernel_9560\3017663775.py:11: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  self.conversation = ConversationChain(


This is a simple chat bot:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Привет, кто ты воин? 
AI:

> Finished chain.
Bot: Я - Афина, ваш личный помощник и виртуальный ассистент.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Приветствую! Я - Афина, ваш личный помощник и виртуальный ассистент.
Human: Ты гречанка полагаю... Почему древняя Греция была побеждена древним Римом? 
AI:

> Finished chain.
Bot: Здравствуйте! Да, я названа в чес

In [13]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Привет
AI:

> Finished chain.
Bot: Здравствуйте! Я ваш личный помощник, готов ответить на любые вопросы и помочь вам. Чем могу быть полезен?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Привет
AI: Здравствуйте! Я ваш личный помощник, готов ответить на любые вопросы и помочь вам. Чем могу быть полезен?
Human: Что такое жизнь? Существует ли жизнь на другой,

Please make a short report abount differences between used memory types

Report:

Here's what I've seen from speaking with it for ~10 minutes:

Simple Memory excels at presicion as all history is kept, good for working on the same topic not for a long time but keeping the precision.

Window Memory probably would be better in something dynamic like quick QA sessions, no need in all history but the latest ones. 

Summary memory misses some details, but generally worked well for not storing info with uncertainty, at least. Not sure why using it if there is a more advanced VectorStoreRetrieverMemory. A tradeoff between precision and high volumes of text.

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [123]:
import os
import requests
from langchain.tools import tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain import hub

giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

@tool
def get_weather(city: str) -> str:
    """
    Get the weather info for a specific city.
    """
    openweather_key = os.environ.get("OPENWEATHER_API_KEY")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        temperature = data['main']['temp']
        description = data['weather'][0]['description']
        
        return f"Погода в {city} {temperature} градусов с {description}."
    except requests.exceptions.RequestException as e:
        return f"Failed to get weather data for {city}. Error: {str(e)}"
    except (KeyError, IndexError):
        return f"Error processing weather data for {city}."

class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        """
        The tool with a language model and an agent function.
        """
        self.llm = llm
        self.agent_function = agent_function

    def run(self, user_input: str) -> str:
        """
        Run the tool with the provided input.
        """
        prompt = hub.pull("hwchase17/openai-functions-agent")

        tools = [self.agent_function]
        agent = create_gigachat_functions_agent(self.llm, tools, prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
        
        result = agent_executor.invoke({"input": user_input})
        return result

Let's check it

In [124]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openweathertool = OpenWeatherAPITool(llm=giga_pro, agent_function=get_weather)
user_input = "Какая погода сейчас в Ставрополе?"
openweathertool.run(user_input)

{'input': 'Какая погода сейчас в Ставрополе?',
 'output': 'Сейчас в Ставрополе 2.16 градуса и облачная погода с прояснениями.'}

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [131]:
import os
import requests
import json
import datetime
from dateutil import parser
from langchain.tools import tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain import hub
import warnings
warnings.filterwarnings("ignore")

giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)
openweather_key = os.environ.get("OPENWEATHER_API_KEY")

def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)

@tool
def get_weather(city: str) -> str:
    """
    Get the weather info for a specific city.
    """
    openweather_key = os.environ.get("OPENWEATHER_API_KEY")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        temperature = data['main']['temp']
        description = data['weather'][0]['description']

        return f"The current temperature in {city} is {temperature}°C with {description}."
    
    except requests.exceptions.RequestException as e:
        return f"Failed to get weather data for {city}. Error: {str(e)}"
    except (KeyError, IndexError):
        return f"Error processing weather data for {city}."

@tool
def get_url_booking_hotels(date_in_str: str, date_out_str: str, city: str) -> str:
    """
    Get the link for booking hotels according to input.
    """
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?'
    params = {
        'adults': '2',
        'checkinDate': date_in.strftime('%Y-%m-%d'),
        'checkoutDate': date_out.strftime('%Y-%m-%d'),
        'childrenAges': '0',
        'geoId': geoid
    }
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your URL for booking: {url} in {city} on {date_in_str} / {date_out_str}'

@tool
def get_url_booking_tickets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> str:
    """
    Get the link for booking tickets according to input.
    """
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?'
    params = {
        'adults_seats': '2',
        'fromId': 'c' + fromid,
        'klass': 'economy',
        'oneway': '2',
        'return_date': date_out.strftime('%Y-%m-%d'),
        'toId': 'c' + toid,
        'when': date_in.strftime('%Y-%m-%d')
    }
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your URL for tickets ordering: {url} from {city_from} to {city_to} on {date_in_str} / {date_out_str}'

class MultiAgent:
    def __init__(self, llm, agent_function_weather, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_function_weather = agent_function_weather
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets

    def _create_agent(self, tools, system_prompt):
        """
        Create an agent with the specified tools and prompt.
        """
        prompt = hub.pull(system_prompt)
        agent = create_gigachat_functions_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=False)

    def run(self, user_input: str):
        """
        Run the multi-agent system for travel planning.
        """
        flight_tool = [self.agent_function_tickets]
        flight_agent = self._create_agent(flight_tool, "hwchase17/openai-functions-agent")
        flight_result = flight_agent.invoke({
            "input": f"Find flights from one city to the second city on given dates according to input: {user_input}"
        })

        hotel_tool = [self.agent_function_hotels]
        hotel_agent = self._create_agent(hotel_tool, "hwchase17/openai-functions-agent")
        hotel_result = hotel_agent.invoke({
            "input": f"Book a hotel in given city on given dates according to input: {user_input}"
        })

        weather_tool = [self.agent_function_weather]
        weather_agent = self._create_agent(weather_tool, "hwchase17/openai-functions-agent")
        weather_result = weather_agent.invoke({
            "input": f"Gather weather data for the city from this request: {user_input}"
        })  
        
        answer = (
            f"""Flight information:
        {flight_result['output'].strip()}

        Hotel information:
        {hotel_result['output'].strip()}

        Weather information:
        {weather_result['output'].strip()}"""
        )
        return answer

In [140]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)
openweather_key = os.environ.get("OPENWEATHER_API_KEY")

traveler = MultiAgent(giga_pro, get_weather, get_url_booking_hotels, get_url_booking_tickets)
user_input = "Организуй поездку из Москвы в Санкт-Петербург на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)

In [141]:
print(answer)

Flight information:
        Here is your URL for ticket ordering: 

https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c213&klass=economy&oneway=2&return_date=2024-11-25&toId=c2&when=2024-11-15 from Москва to Санкт-Петербург on 15-11-2024 / 25-11-2024.

        Hotel information:
        Here is your URL for booking: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2 in Санкт-Петербург on 15-11-2024 / 25-11-2024.

        Weather information:
        Here's the weather forecast for Saint Petersburg, Russia, from November 15 to 24, 2024:

**November 15 - 24, 2024:**

- **Daytime Temperature:** 5.7°C on average
- **Nighttime Temperature:** 0.9°C on average
- **Precipitation:** Light rain expected on several days
- **Cloud Cover:** Partly cloudy to overcast skies
- **Wind Speed:** Moderate winds of around 11 km/h

Have a pleasant trip!
